<a href="https://colab.research.google.com/github/halnegheimish/ForcedInvalidation/blob/main/notebooks/MNLI_FI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Code in this notebook is based on https://github.com/huggingface/notebooks/blob/6ca682955173cc9d36ffa431ddda505a048cbe80/examples/text_classification.ipynb

# preliminaries

In [ ]:
! pip install datasets transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 91.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 38.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 70.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 10.3 MB/s eta 0:00:00


In [ ]:
!apt install subversion

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libapr1 libaprutil1 libserf-1-1 libsvn1 libutf8proc2
Suggested packages:
  db5.3-util libapache2-mod-svn subversion-tools
The following NEW packages will be installed:
  libapr1 libaprutil1 libserf-1-1 libsvn1 libutf8proc2 subversion
0 upgraded, 6 newly installed, 0 to remove and 24 not upgraded.
Need to get 2,355 kB of archives.
After this operation, 10.3 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/main amd64 libapr1 amd64 1.6.5-1ubuntu1 [91.4 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libaprutil1 amd64 1.6.1-4ubuntu2.1 [84.9 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal/universe amd64 libserf-1-1 amd64 1.3.9-8build1 [45.2 kB]
Get:4 http://archive.ubuntu.com/ubuntu focal/universe amd64 libutf8proc2 amd64 2.5.0-1 [50.0 kB]
Get:5 http://archive.ubuntu.com/ubuntu focal-

In [ ]:
!mkdir data

In [ ]:
cd data

In [ ]:
!svn checkout https://github.com/halnegheimish/ForcedInvalidation/trunk/data/eval/mnli/
!svn checkout https://github.com/halnegheimish/ForcedInvalidation/trunk/data/eval/anli/


A    mnli/mnli_dev_1gram_h
A    mnli/mnli_dev_1gram_h/dataset.arrow
A    mnli/mnli_dev_1gram_h/dataset_info.json
A    mnli/mnli_dev_1gram_h/state.json
A    mnli/mnli_dev_1gram_p
A    mnli/mnli_dev_1gram_p/dataset.arrow
A    mnli/mnli_dev_1gram_p/dataset_info.json
A    mnli/mnli_dev_1gram_p/state.json
A    mnli/mnli_dev_2gram_h
A    mnli/mnli_dev_2gram_h/dataset.arrow
A    mnli/mnli_dev_2gram_h/dataset_info.json
A    mnli/mnli_dev_2gram_h/state.json
A    mnli/mnli_dev_2gram_p
A    mnli/mnli_dev_2gram_p/dataset.arrow
A    mnli/mnli_dev_2gram_p/dataset_info.json
A    mnli/mnli_dev_2gram_p/state.json
A    mnli/mnli_dev_3gram_h
A    mnli/mnli_dev_3gram_h/dataset.arrow
A    mnli/mnli_dev_3gram_h/dataset_info.json
A    mnli/mnli_dev_3gram_h/state.json
A    mnli/mnli_dev_3gram_p
A    mnli/mnli_dev_3gram_p/dataset.arrow
A    mnli/mnli_dev_3gram_p/dataset_info.json
A    mnli/mnli_dev_3gram_p/state.json
A    mnli/mnli_mm_dev_1gram_h
A    mnli/mnli_mm_dev_1gram_h/dataset.arrow
A    mnli/mnli_mm_de

In [ ]:
!gdown --fuzzy --folder https://drive.google.com/drive/folders/1UuqBfHn2sOu7KyCajegtuE6gfXLCsvIc?usp=share_link

Retrieving folder list
Retrieving folder 14Yp4FzHmi1iLYAGGppbHxjbY_K8_PoVd mnli_train_ngrams
Processing file 14iassBk0MT57VPvUvMGDoGE5NG40PyB0 dataset_info.json
Processing file 14hykXzJHf-G_v0s4gafb6NvpZjaHQOHr dataset.arrow
Processing file 14j4j-lPNHnWhC5L7eX6yaXW3--xvqFX5 state.json
Retrieving folder 142PDErbszHUGtSKmg8PnhoRBa9AbDMh6 mnli_val_ngrams
Processing file 145bcFTzktpEeBxX9vUBVRE_rHk-l6Wfh dataset_info.json
Processing file 14JYCMFppsqqwjohQ1KZg2ifB6T8RhsYG dataset.arrow
Processing file 149c33JR6OPBl3CPknYr5DMk3F7tebs7W state.json
Retrieving folder list completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=14iassBk0MT57VPvUvMGDoGE5NG40PyB0
To: /content/mnli/mnli_train_ngrams/dataset_info.json
100% 2.37k/2.37k [00:00<00:00, 5.92MB/s]
Downloading...
From: https://drive.google.com/uc?id=14hykXzJHf-G_v0s4gafb6NvpZjaHQOHr
To: /content/mnli/mnli_train_ngrams/dataset.arrow
100% 138M/138M [00:01<00:00, 105MB/

In [ ]:
cd ..

In [ ]:
import itertools
import random
import numpy as np
import torch
import pandas as pd

seed= 42
random.seed(seed)
np.random.seed(seed)

from transformers import set_seed

set_seed(seed)


In [ ]:
from datasets import load_dataset, load_metric

metric = load_metric('glue', 'mnli')

<ipython-input-6-df967808ea48>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('glue', 'mnli')


#train model

In [ ]:
from datasets import load_from_disk

In [ ]:
ngrams_train = load_from_disk('data/mnli/mnli_train_ngrams')

In [ ]:
ngrams_dev= load_from_disk('data/mnli/mnli_val_ngrams')

In [ ]:
model_checkpoint = "bert-base-uncased"
batch_size = 16

In [ ]:
task='mnli'

In [ ]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

In [ ]:
task_to_keys = {
    "cola": ("sentence", None),
    "mnli": ("premise", "hypothesis"),
    "mnli-mm": ("premise", "hypothesis"),
    "mrpc": ("sentence1", "sentence2"),
    "qnli": ("question", "sentence"),
    "qqp": ("question1", "question2"),
    "rte": ("sentence1", "sentence2"),
    "sst2": ("sentence", None),
    "stsb": ("sentence1", "sentence2"),
    "wnli": ("sentence1", "sentence2"),
}

In [ ]:
sentence1_key, sentence2_key = task_to_keys[task]
def preprocess_function(examples):
    if sentence2_key is None:
        return tokenizer(examples[sentence1_key], truncation=True)
    return tokenizer(examples[sentence1_key], examples[sentence2_key], truncation=True)

In [ ]:
encoded_train = ngrams_train.map(preprocess_function, batched=True)

Map:   0%|          | 0/677742 [00:00<?, ? examples/s]

In [ ]:
encoded_val = ngrams_dev.map(preprocess_function, batched=True)

Map:   0%|          | 0/75240 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

num_labels = 4 
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
metric_name = "pearson" if task == "stsb" else "matthews_correlation" if task == "cola" else "accuracy"
model_name = model_checkpoint.split("/")[-1]

args = TrainingArguments(
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    output_dir=f'bert_base_finetuned_mnli_ngrams_{seed}'
)

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    if task != "stsb":
        predictions = np.argmax(predictions, axis=1)
    else:
        predictions = predictions[:, 0]
    return  metric.compute(predictions=predictions, references=labels)

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_train,
    eval_dataset=encoded_val,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.317000,0.299368,0.890936
2,0.231900,0.302405,0.898312
3,0.175800,0.352762,0.900532
4,0.132600,0.444428,0.900439


Epoch,Training Loss,Validation Loss,Accuracy
1,0.317000,0.299368,0.890936
2,0.231900,0.302405,0.898312
3,0.175800,0.352762,0.900532
4,0.132600,0.444428,0.900439


#evaluation

In [ ]:
datasets=[dict(path='data/mnli/mnli_mm_dev_1gram_p' , name='mnli_mm_1g_p', split='mismatched'),
          dict(path='data/mnli/mnli_mm_dev_2gram_p' , name='mnli_mm_2g_p', split='mismatched'),
          dict(path='data/mnli/mnli_mm_dev_3gram_p' , name='mnli_mm_3g_p', split='mismatched'),
          dict(path='data/mnli/mnli_mm_dev_1gram_h' , name='mnli_mm_1g_h', split='mismatched'),
          dict(path='data/mnli/mnli_mm_dev_2gram_h' , name='mnli_mm_2g_h', split='mismatched'),
          dict(path='data/mnli/mnli_mm_dev_3gram_h' , name='mnli_mm_3g_h', split='mismatched'),
          dict(path='data/mnli/mnli_dev_1gram_p' , name='mnli_1g_p', split='matched'),
          dict(path='data/mnli/mnli_dev_2gram_p' , name='mnli_2g_p', split='matched'),
          dict(path='data/mnli/mnli_dev_3gram_p' , name='mnli_3g_p', split='matched'),
          dict(path='data/mnli/mnli_dev_1gram_h' , name='mnli_1g_h', split='matched'),
          dict(path='data/mnli/mnli_dev_2gram_h' , name='mnli_2g_h', split='matched'),
          dict(path='data/mnli/mnli_dev_3gram_h' , name='mnli_3g_h', split='matched'),
          dict(path='data/anli/anli_dev_r1_1gram_p' , name='anli_r1_1g_p', split='anli_r1'),
          dict(path='data/anli/anli_dev_r1_2gram_p' , name='anli_r1_2g_p', split='anli_r1'),
          dict(path='data/anli/anli_dev_r1_3gram_p' , name='anli_r1_3g_p', split='anli_r1'),
          dict(path='data/anli/anli_dev_r1_1gram_h' , name='anli_r1_1g_h', split='anli_r1'),
          dict(path='data/anli/anli_dev_r1_2gram_h' , name='anli_r1_2g_h', split='anli_r1'),
          dict(path='data/anli/anli_dev_r1_3gram_h' , name='anli_r1_3g_h', split='anli_r1'),
          dict(path='data/anli/anli_dev_r2_1gram_p' , name='anli_r2_1g_p', split='anli_r2'),
          dict(path='data/anli/anli_dev_r2_2gram_p' , name='anli_r2_2g_p', split='anli_r2'),
          dict(path='data/anli/anli_dev_r2_3gram_p' , name='anli_r2_3g_p', split='anli_r2'),
          dict(path='data/anli/anli_dev_r2_1gram_h' , name='anli_r2_1g_h', split='anli_r2'),
          dict(path='data/anli/anli_dev_r2_2gram_h' , name='anli_r2_2g_h', split='anli_r2'),
          dict(path='data/anli/anli_dev_r2_3gram_h' , name='anli_r2_3g_h', split='anli_r2'),
          dict(path='data/anli/anli_dev_r3_1gram_p' , name='anli_r3_1g_p', split='anli_r3'),
          dict(path='data/anli/anli_dev_r3_2gram_p' , name='anli_r3_2g_p', split='anli_r3'),
          dict(path='data/anli/anli_dev_r3_3gram_p' , name='anli_r3_3g_p', split='anli_r3'),
          dict(path='data/anli/anli_dev_r3_1gram_h' , name='anli_r3_1g_h', split='anli_r3'),
          dict(path='data/anli/anli_dev_r3_2gram_h' , name='anli_r3_2g_h', split='anli_r3'),
          dict(path='data/anli/anli_dev_r3_3gram_h' , name='anli_r3_3g_h', split='anli_r3'), 
    ]

runs=[dict(path='bert_base_finetuned_mnli_ngrams_42/checkpoint-211795/',
           name='invalid', seed=42),
      ]

In [ ]:
def load_model(path):
  tokenizer = AutoTokenizer.from_pretrained(path)
  model = AutoModelForSequenceClassification.from_pretrained(path)
  model_checkpoint=path
  model_name = model_checkpoint.split("/")[-2]
  args = TrainingArguments(
    f"{model_name}",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    do_train= False,
    do_eval=  True,
    per_device_eval_batch_size=16,
  )
  return tokenizer, model, args


def eval_dataset(dataset, tokenizer, model, args):
  encoded_dataset= dataset.map(preprocess_function, batched=True)
  trainer = Trainer(
    model,
    args,
    train_dataset=None,
    eval_dataset=encoded_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
  ) 
  metrics= trainer.evaluate()
  return metrics

def eval_dataset(dataset, tokenizer, model, args):
  encoded_dataset= dataset.map(preprocess_function, batched=True)
  trainer = Trainer(
    model,
    args,
    train_dataset=None,
    eval_dataset=encoded_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
  ) 
  metrics= trainer.evaluate()
  return metrics

def eval_mnli(tokenizer, model, args):
  dataset= load_dataset('glue', 'mnli')['validation_matched']
  encoded_dataset= dataset.map(preprocess_function, batched=True)
  trainer = Trainer(
    model,
    args,
    train_dataset=None,
    eval_dataset=encoded_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
  ) 
  metrics= trainer.evaluate()
  return metrics

def eval_mnli_mm(tokenizer, model, args):
  dataset= load_dataset('glue', 'mnli')['validation_mismatched']
  encoded_dataset= dataset.map(preprocess_function, batched=True)
  trainer = Trainer(
    model,
    args,
    train_dataset=None,
    eval_dataset=encoded_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
  ) 
  metrics= trainer.evaluate()
  return metrics

def eval_anli(tokenizer, model, args, split='r3'):
  dataset= load_dataset('anli')[f'dev_{split}']
  encoded_dataset= dataset.map(preprocess_function, batched=True)
  trainer = Trainer(
    model,
    args,
    train_dataset=None,
    eval_dataset=encoded_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
  ) 
  metrics= trainer.evaluate()
  return metrics

In [ ]:
results_all={}
for run in runs:
  tokenizer, model, args= load_model(run['path'])
  results_all[f'seed_{run["seed"]}']={}
  results_all[f'seed_{run["seed"]}']['dev_matched']=eval_mnli(tokenizer, model, args)['eval_accuracy']
  results_all[f'seed_{run["seed"]}']['dev_mismatched']=eval_mnli_mm(tokenizer, model, args)['eval_accuracy']
  results_all[f'seed_{run["seed"]}']['dev_anli_r1']=eval_anli(tokenizer, model, args, split='r1')['eval_accuracy']
  results_all[f'seed_{run["seed"]}']['dev_anli_r2']=eval_anli(tokenizer, model, args, split='r2')['eval_accuracy']
  results_all[f'seed_{run["seed"]}']['dev_anli_r3']=eval_anli(tokenizer, model, args, split='r3')['eval_accuracy']
  for dp in datasets:
    dataset= load_from_disk(dp['path'])
    acc= eval_dataset(dataset, tokenizer, model, args)['eval_accuracy']
    results_all[f'seed_{run["seed"]}'][dp['name']]=acc




Generating train split:   0%|          | 0/392702 [00:00<?, ? examples/s]

Generating validation_matched split:   0%|          | 0/9815 [00:00<?, ? examples/s]

Generating validation_mismatched split:   0%|          | 0/9832 [00:00<?, ? examples/s]

Generating test_matched split:   0%|          | 0/9796 [00:00<?, ? examples/s]

Generating test_mismatched split:   0%|          | 0/9847 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/1199 [00:00<?, ? examples/s]

Map:   0%|          | 0/1199 [00:00<?, ? examples/s]

Map:   0%|          | 0/1199 [00:00<?, ? examples/s]

Map:   0%|          | 0/1199 [00:00<?, ? examples/s]

Map:   0%|          | 0/1199 [00:00<?, ? examples/s]

In [ ]:
df=pd.DataFrame(results_all)

In [ ]:
df

,seed_42
anli_r1_1g_h,0.001001
anli_r1_1g_p,0.000000
anli_r1_2g_h,0.006006
anli_r1_2g_p,0.000000
anli_r1_3g_h,0.015015
anli_r1_3g_p,0.001001
anli_r2_1g_h,0.002002
anli_r2_1g_p,0.000000
anli_r2_2g_h,0.010010
anli_r2_2g_p,0.000000
